https://github.com/studioimaginaire/phue 

Phue is a package for interacting with the Hue hub. 

I have 2 remotes, 1 tap, and 3 motion (+ temp and light) sensors. Hue also creates 'sensors' for presence ('HomeAway', 'Robins iPhone') as well as dimmer secnes.

In [1]:
from phue import Bridge

In [2]:
bridge_ip = '192.168.0.3'

In [3]:
b = Bridge(bridge_ip)

# Sensors
Sensors are more simple than lights in some ways, as lights can be in groups. A significant part of the HA lights component is dealing with groups. Instead we have the issue that Hue treates a motion sensor device as 3 seperate sensors consisting of motion, temperature and light level. I prefer to group by device so need to have a routine which correctly ties sensors to a device.

In [4]:
response = b.get_sensor_objects('name')

get_sensor_objects() is a method on the bridge object which requests data for ALL sensors.

Lets request the response by sensor name - we are returned a list of sensor objects indexed by name

In [5]:
response

{'Bedroom sensor': <phue.Sensor object "Bedroom sensor" at 0x1110d07f0>,
 'Daylight': <phue.Sensor object "Daylight" at 0x1110e3a58>,
 'Dimmer Switch 2 SceneCycle': <phue.Sensor object "Dimmer Switch 2 SceneCycle" at 0x1110e3b38>,
 'Dimmer Switch 20 SceneCycle': <phue.Sensor object "Dimmer Switch 20 SceneCycle" at 0x1110f5e10>,
 'Dimmer Switch 22 SceneCycle': <phue.Sensor object "Dimmer Switch 22 SceneCycle" at 0x1110f5e80>,
 'Hall Sensor': <phue.Sensor object "Hall Sensor" at 0x1110e3ac8>,
 'Hall remote': <phue.Sensor object "Hall remote" at 0x1110f5e48>,
 'HomeAway': <phue.Sensor object "HomeAway" at 0x1110f5cc0>,
 'Hue Tap': <phue.Sensor object "Hue Tap" at 0x1110f5eb8>,
 'Hue ambient light sensor 1': <phue.Sensor object "Hue ambient light sensor 1" at 0x1110e3be0>,
 'Hue ambient light sensor 2': <phue.Sensor object "Hue ambient light sensor 2" at 0x1110f5c18>,
 'Hue ambient light sensor 3': <phue.Sensor object "Hue ambient light sensor 3" at 0x1110f5d68>,
 'Hue temperature sensor 1

Print the attributes of a sensor

In [6]:
def print_sensor_attrs(sensor):
    """Helper to print sensor attributes. pass a sensor name."""
    assert type(sensor) == str
    print("name : {}".format(response[sensor].name))
    print("state : {}".format(response[sensor].state))
    print("type : {}".format(response[sensor].type))
    print("manufacturername : {}".format(response[sensor].manufacturername))
    print("modelid : {}".format(response[sensor].modelid))
    print("uniqueid : {}".format(response[sensor].uniqueid))
    print("sensor_id : {}".format(response[sensor].sensor_id))
    print("config : {}".format(response[sensor].config))
    return
    
print_sensor_attrs('Bedroom sensor')

name : Bedroom sensor
state : {'presence': False, 'lastupdated': '2018-01-18T07:04:35'}
type : ZLLPresence
manufacturername : Philips
modelid : SML001
uniqueid : 00:17:88:01:02:00:b5:ce-02-0406
sensor_id : 9
config : {'on': True, 'battery': 100, 'reachable': True, 'alert': 'none', 'ledindication': False, 'usertest': False, 'sensitivity': 2, 'sensitivitymax': 2, 'pending': []}


In [7]:
str(type(response['Hue Tap']))

"<class 'phue.Sensor'>"

lets create a unique device id, where for a motion sensor the device includes both temperature and light level sensors

In [8]:
def device_id(sensor_obj):
    """Helper that takes a sensor object and returns its unique device id."""
    assert str(type(sensor_obj)) == "<class 'phue.Sensor'>"
    return sensor_obj.uniqueid.split(':')[-1][0:5]

In [9]:
device_id(response['Bedroom sensor'])

'ce-02'

In [10]:
device_id(response['Hue ambient light sensor 2'])

'ce-02'

In [11]:
device_id(response['Hue temperature sensor 2'])

'ce-02'

In [12]:
print_sensor_attrs('Robins iPhone')

name : Robins iPhone
state : {'presence': False, 'lastupdated': '2018-01-17T09:26:14'}
type : Geofence
manufacturername : Philips
modelid : HA_GEOFENCE
uniqueid : L_02_IYwHM
sensor_id : 12
config : {'on': True, 'reachable': True}


In [13]:
print_sensor_attrs('Bedroom sensor')

name : Bedroom sensor
state : {'presence': False, 'lastupdated': '2018-01-18T07:04:35'}
type : ZLLPresence
manufacturername : Philips
modelid : SML001
uniqueid : 00:17:88:01:02:00:b5:ce-02-0406
sensor_id : 9
config : {'on': True, 'battery': 100, 'reachable': True, 'alert': 'none', 'ledindication': False, 'usertest': False, 'sensitivity': 2, 'sensitivitymax': 2, 'pending': []}


print_sensor_attrs('Daylight') # Keeps trakc of whether the sun is up or not

In [14]:
print_sensor_attrs('HomeAway') # Not sure what this is

name : HomeAway
state : {'presence': False, 'lastupdated': '2018-01-17T09:26:14'}
type : CLIPPresence
manufacturername : Philips
modelid : HOMEAWAY
uniqueid : L_01_9rQ8A
sensor_id : 13
config : {'on': True, 'reachable': True}


In [15]:
print_sensor_attrs('Hall remote')

name : Hall remote
state : {'buttonevent': 4002, 'lastupdated': '2018-01-15T19:37:31'}
type : ZLLSwitch
manufacturername : Philips
modelid : RWL021
uniqueid : 00:17:88:01:02:c0:e1:9c-02-fc00
sensor_id : 22
config : {'on': True, 'battery': 100, 'reachable': True, 'pending': []}


id and names can be related using the method get_sensor_id_by_name()

In [16]:
b.get_sensor_id_by_name('Hall remote')

'22'

Lets check what response by id returns - sensors are indexed 1-24 as they appear listed from the sensors API return

In [17]:
response_id = b.get_sensor_objects('id')
response_id

{1: <phue.Sensor object "Daylight" at 0x1110e3a58>,
 2: <phue.Sensor object "Remote bedroom" at 0x1110e3b00>,
 3: <phue.Sensor object "Dimmer Switch 2 SceneCycle" at 0x1110e3b38>,
 4: <phue.Sensor object "Hue temperature sensor 1" at 0x1110e3b70>,
 5: <phue.Sensor object "Hall Sensor" at 0x1110e3ac8>,
 6: <phue.Sensor object "Hue ambient light sensor 1" at 0x1110e3be0>,
 7: <phue.Sensor object "MotionSensor 5.Companion" at 0x1110d0828>,
 8: <phue.Sensor object "Hue temperature sensor 2" at 0x1110d0ba8>,
 9: <phue.Sensor object "Bedroom sensor" at 0x1110d07f0>,
 10: <phue.Sensor object "Hue ambient light sensor 2" at 0x1110f5c18>,
 11: <phue.Sensor object "MotionSensor 9.Companion" at 0x1110f5c50>,
 12: <phue.Sensor object "Robins iPhone" at 0x1110f5c88>,
 13: <phue.Sensor object "HomeAway" at 0x1110f5cc0>,
 14: <phue.Sensor object "Hue temperature sensor 3" at 0x1110f5cf8>,
 15: <phue.Sensor object "Living room sensor" at 0x1110f5d30>,
 16: <phue.Sensor object "Hue ambient light sensor

Probably accessing sensors via id is more useful than name. I need an init method that groups id by devices

# Lights

The HA component [adds the wrapper class](https://github.com/home-assistant/home-assistant/blob/aad14b8b87e23f61300f31b96b8a92769527516c/homeassistant/components/light/hue.py#L210) HueLight(Light).

Lights can be in groups and groups have info. I don't have any groups so access light objects directly

In [18]:
b.lights

[<phue.Light object "Hallway" at 0x1110a32b0>,
 <phue.Light object "Bedroom light" at 0x1111104a8>,
 <phue.Light object "Lamp" at 0x111110908>,
 <phue.Light object "Kitchen light" at 0x1111106d8>]

In [19]:
def print_light_attrs(light_index):
    """Helper to print light attributes. Pass the index of light"""
    assert type(light_index) == int
    print("name : {}".format(b.lights[light_index].name))
    print("brightness : {}".format(b.lights[light_index].brightness))
    print("on : {}".format(b.lights[light_index].on))
    
light_index = 0
print_light_attrs(light_index)

name : Hallway
brightness : 1
on : False
